### Cell 1 - Locate inputs and set folders

입력/출력 폴더 경로를 고정하고 파일명을 정의한다


In [4]:

from pathlib import Path
import os

BASE = Path.cwd()
DATA_RAW = BASE / "data_raw"
DATA_PROC = BASE / "data_processed"
MAP_DIR = BASE / "mapping"

for p in [DATA_RAW, DATA_PROC, MAP_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# Update these if your filenames differ
PATH_0513 = DATA_RAW / "crime_2005_2013_raw.csv"
PATH_1424 = DATA_RAW / "crime_2014_2024_raw_as_is.csv"

print("PATH_0513:", PATH_0513)
print("PATH_1424:", PATH_1424)
print("DATA_PROC:", DATA_PROC)
print("MAP_DIR:", MAP_DIR)


PATH_0513: C:\Users\82102\Desktop\Structural Shifts in International Crime_South Korea (2005–2025)\notebooks\data_raw\crime_2005_2013_raw.csv
PATH_1424: C:\Users\82102\Desktop\Structural Shifts in International Crime_South Korea (2005–2025)\notebooks\data_raw\crime_2014_2024_raw_as_is.csv
DATA_PROC: C:\Users\82102\Desktop\Structural Shifts in International Crime_South Korea (2005–2025)\notebooks\data_processed
MAP_DIR: C:\Users\82102\Desktop\Structural Shifts in International Crime_South Korea (2005–2025)\notebooks\mapping


### Cell 2 - Load 2005–2013 raw CSV

원본(2005–2013) 파일을 로드하고 기본 구조를 확인한다


In [7]:

import pandas as pd

if not PATH_0513.exists():
    raise FileNotFoundError(f"Missing file: {PATH_0513}")

raw_0513 = pd.read_csv(PATH_0513, header=None, dtype=str, encoding="utf-8", engine="python")
print("RAW_0513 SHAPE:", raw_0513.shape)
display(raw_0513.head(8))


RAW_0513 SHAPE: (202, 66)


,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,범죄별(1),범죄별(2),범죄별(3),2005,2005.1,2005.2,2005.3,2005.4,2005.5,2005.6,...,2012.4,2012.5,2012.6,2013,2013.1,2013.2,2013.3,2013.4,2013.5,2013.6
1,범죄별(1),범죄별(2),범죄별(3),발생건수 (건),발생비 (%),검거건수 (건),검거율 (%),검거인원 (명),남자검거인원 (명),여자검거인원 (명),...,검거인원 (명),남자검거인원 (명),여자검거인원 (명),발생건수 (건),발생비 (%),검거건수 (건),검거율 (%),검거인원 (명),남자검거인원 (명),여자검거인원 (명)
2,합계,소계,소계,1893896,3882.3,1624522,85.8,1897093,1697448,199645,...,1983697,1628843,354854,2006682,3921,1543930,76.9,1996629,1646783,349846
3,형법범,소계,소계,825840,1692.9,634244,76.8,856072,741461,114611,...,1035335,843596,191739,1057855,2067,696449,65.8,1027127,841071,186056
4,NaN,재산범죄,소계,442015,906.1,285864,64.7,339896,297882,42014,...,467497,373015,94482,671224,1311,351361,52.3,489493,392618,96875
5,NaN,NaN,절도,191114,391.8,81146,42.5,106105,95908,10197,...,108199,87053,21146,290841,568,119696,41.2,106608,85259,21349
6,NaN,NaN,장물,3547,7.3,3490,98.4,4091,3857,234,...,5517,4919,598,6491,13,6248,96.3,7465,6804,661
7,NaN,NaN,사기,203697,417.6,167027,82,189627,162129,27498,...,279334,218419,60915,274086,536,169318,61.8,297713,235466,62247


### Cell 3 - Detect header row and build df_0513

헤더가 포함된 행을 탐지하고 정상 컬럼명을 가진 df_0513를 만든다


In [8]:

import numpy as np
import re

# Find a row containing '범죄별' and a year token
header_idx = None
for i in range(min(50, len(raw_0513))):
    row = raw_0513.iloc[i].astype(str).tolist()
    joined = " ".join(row)
    if ("범죄별" in joined) and re.search(r"2005", joined):
        header_idx = i
        break

if header_idx is None:
    raise ValueError("Header row not found for 2005–2013 file.")

header = raw_0513.iloc[header_idx].tolist()
df_0513 = raw_0513.iloc[header_idx+1:].copy()
df_0513.columns = header
df_0513 = df_0513.reset_index(drop=True)

print("HEADER IDX:", header_idx)
print("df_0513 SHAPE:", df_0513.shape)
print("df_0513 columns (first 20):", list(df_0513.columns)[:20])
display(df_0513.head(10))


HEADER IDX: 0
df_0513 SHAPE: (201, 66)
df_0513 columns (first 20): ['범죄별(1)', '범죄별(2)', '범죄별(3)', '2005', '2005.1', '2005.2', '2005.3', '2005.4', '2005.5', '2005.6', '2006', '2006.1', '2006.2', '2006.3', '2006.4', '2006.5', '2006.6', '2007', '2007.1', '2007.2']


,범죄별(1),범죄별(2),범죄별(3),2005,2005.1,2005.2,2005.3,2005.4,2005.5,2005.6,...,2012.4,2012.5,2012.6,2013,2013.1,2013.2,2013.3,2013.4,2013.5,2013.6
0,범죄별(1),범죄별(2),범죄별(3),발생건수 (건),발생비 (%),검거건수 (건),검거율 (%),검거인원 (명),남자검거인원 (명),여자검거인원 (명),...,검거인원 (명),남자검거인원 (명),여자검거인원 (명),발생건수 (건),발생비 (%),검거건수 (건),검거율 (%),검거인원 (명),남자검거인원 (명),여자검거인원 (명)
1,합계,소계,소계,1893896,3882.3,1624522,85.8,1897093,1697448,199645,...,1983697,1628843,354854,2006682,3921,1543930,76.9,1996629,1646783,349846
2,형법범,소계,소계,825840,1692.9,634244,76.8,856072,741461,114611,...,1035335,843596,191739,1057855,2067,696449,65.8,1027127,841071,186056
3,NaN,재산범죄,소계,442015,906.1,285864,64.7,339896,297882,42014,...,467497,373015,94482,671224,1311,351361,52.3,489493,392618,96875
4,NaN,NaN,절도,191114,391.8,81146,42.5,106105,95908,10197,...,108199,87053,21146,290841,568,119696,41.2,106608,85259,21349
5,NaN,NaN,장물,3547,7.3,3490,98.4,4091,3857,234,...,5517,4919,598,6491,13,6248,96.3,7465,6804,661
6,NaN,NaN,사기,203697,417.6,167027,82,189627,162129,27498,...,279334,218419,60915,274086,536,169318,61.8,297713,235466,62247
7,NaN,NaN,횡령,25412,52.1,21677,85.3,24773,22347,2426,...,35112,28561,6551,36214,71,24626,68,37886,30738,7148
8,NaN,NaN,배임,5901,12.1,5143,87.2,7243,6402,841,...,8897,7271,1626,5296,10,4500,85,9734,7942,1792
9,NaN,NaN,손괴,12344,25.3,7381,59.8,8057,7239,818,...,30438,26792,3646,58296,114,26973,46.3,30087,26409,3678


### Cell 4 - Select incident-count year columns (2005–2013)

발생건수에 해당하는 연도 컬럼(YYYY)만 남기고, 비율/검거 컬럼을 제외한다


In [9]:

import pandas as pd
import re

# Keep only pure year columns like '2005'...'2013'
year_cols = [c for c in df_0513.columns if re.fullmatch(r"(200[5-9]|201[0-3])", str(c).strip())]
if not year_cols:
    raise ValueError("No incident-count year columns (YYYY) detected. Check header parsing.")

cat_cols = [c for c in ["범죄별(1)", "범죄별(2)", "범죄별(3)"] if c in df_0513.columns]
if not cat_cols:
    raise ValueError("No category columns found among 범죄별(1)/(2)/(3).")

keep_cols = cat_cols + year_cols
df_0513_clean = df_0513[keep_cols].copy()

print("CATEGORY COLS:", cat_cols)
print("YEAR COLS:", year_cols)
display(df_0513_clean.head(10))


CATEGORY COLS: ['범죄별(1)', '범죄별(2)', '범죄별(3)']
YEAR COLS: ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']


,범죄별(1),범죄별(2),범죄별(3),2005,2006,2007,2008,2009,2010,2011,2012,2013
0,범죄별(1),범죄별(2),범죄별(3),발생건수 (건),발생건수 (건),발생건수 (건),발생건수 (건),발생건수 (건),발생건수 (건),발생건수 (건),발생건수 (건),발생건수 (건)
1,합계,소계,소계,1893896,1829211,1965977,2189452,2168185,1917300,1902720,1944906,2006682
2,형법범,소계,소계,825840,828021,845311,897536,993136,939171,997263,1038609,1057855
3,NaN,재산범죄,소계,442015,455948,469654,503302,561972,568623,605360,637686,671224
4,NaN,NaN,절도,191114,190745,212530,223264,256680,268007,281561,293074,290841
5,NaN,NaN,장물,3547,2432,3050,2212,3381,3206,2606,3856,6491
6,NaN,NaN,사기,203697,203346,186115,205140,224889,205913,226359,241275,274086
7,NaN,NaN,횡령,25412,25084,24122,26750,27362,26312,27882,33044,36214
8,NaN,NaN,배임,5901,5402,5256,5135,6709,14619,5401,5176,5296
9,NaN,NaN,손괴,12344,28939,38581,40801,42951,50566,61551,61261,58296


### Cell 5 - Rebuild long_0513 with hierarchical forward-fill

계층형 분류를 ffill로 복구하고 long format으로 변환한다


In [10]:

import numpy as np
import pandas as pd

work = df_0513_clean.copy()
cat_cols = [c for c in ["범죄별(1)", "범죄별(2)", "범죄별(3)"] if c in work.columns]
year_cols = [c for c in work.columns if c not in cat_cols]

# 1) forward-fill hierarchy
work[cat_cols] = work[cat_cols].ffill()

# 2) drop grand total only (do NOT drop '소계' because it is meaningful rows in this dataset)
def is_total(row):
    vals = [str(row[c]).strip() for c in cat_cols if pd.notna(row[c])]
    return any(v == "합계" for v in vals)

work = work.loc[~work.apply(is_total, axis=1)].copy()

# 3) build category_raw from all available category columns
def build_cat(row):
    parts = []
    for c in cat_cols:
        v = row.get(c, None)
        if pd.notna(v):
            vv = str(v).strip()
            if vv and vv.lower() not in ["nan"]:
                parts.append(vv)
    return " / ".join(parts)

work["category_raw"] = work.apply(build_cat, axis=1)

# 4) wide -> long
long_0513 = work.melt(
    id_vars=["category_raw"],
    value_vars=year_cols,
    var_name="year",
    value_name="value"
)
long_0513["year"] = long_0513["year"].astype(int)
long_0513["value"] = pd.to_numeric(long_0513["value"], errors="coerce").fillna(0)

print("long_0513:", long_0513.shape, "| years:", long_0513["year"].min(), "-", long_0513["year"].max())
print("sample categories:")
display(long_0513["category_raw"].drop_duplicates().head(15))

# sanity: drug keywords should exist
drug_sample = [x for x in long_0513["category_raw"].drop_duplicates().tolist() if "마약" in x]
print("categories containing '마약':", len(drug_sample))
print(drug_sample[:10])


long_0513: (1800, 3) | years: 2005 - 2013
sample categories:


0     범죄별(1) / 범죄별(2) / 범죄별(3)
1                형법범 / 소계 / 소계
2              형법범 / 재산범죄 / 소계
3              형법범 / 재산범죄 / 절도
4              형법범 / 재산범죄 / 장물
5              형법범 / 재산범죄 / 사기
6              형법범 / 재산범죄 / 횡령
7              형법범 / 재산범죄 / 배임
8              형법범 / 재산범죄 / 손괴
9          형법범 / 강력범죄(흉악) / 소계
10         형법범 / 강력범죄(흉악) / 살인
11         형법범 / 강력범죄(흉악) / 강도
12         형법범 / 강력범죄(흉악) / 방화
13         형법범 / 강력범죄(흉악) / 강간
14         형법범 / 강력범죄(폭력) / 소계
Name: category_raw, dtype: object

categories containing '마약': 4
['특별법범 / 마약류관리에관한법률（대마） / 소계', '특별법범 / 마약류관리에관한법률（마약） / 소계', '특별법범 / 마약류관리에관한법률（향정） / 소계', '특별법범 / 마약류불법거래방지에관한특례법 / 소계']


### Cell 6 - Load 2014–2024 rectified CSV and reshape to long

정리된(헤더 정상화) 2014–2024 파일을 long format으로 변환한다


In [12]:
print("COLUMNS SAMPLE (first 30):")
print(list(df_1424.columns)[:30])

print("\nCOLUMNS CONTAINING '201':")
print([c for c in df_1424.columns if "201" in str(c)])

print("\nCOLUMNS CONTAINING '202':")
print([c for c in df_1424.columns if "202" in str(c)])


COLUMNS SAMPLE (first 30):
['○ 범죄의 발생 검거상황(총괄) []', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13']

COLUMNS CONTAINING '201':
[]

COLUMNS CONTAINING '202':
[]


In [13]:
import pandas as pd
import re

# 입력 파일 선택 (rectified가 있으면 우선)
RECT_1424 = DATA_RAW / "crime_2014_2024_raw_rectified_header.csv"
input_1424 = RECT_1424 if RECT_1424.exists() else PATH_1424

if not input_1424.exists():
    raise FileNotFoundError(f"Missing file: {input_1424}")

# 1) header=None으로 raw 로드 (중요)
raw_1424 = pd.read_csv(input_1424, header=None, dtype=str, encoding="utf-8", engine="python")
print("raw_1424 shape:", raw_1424.shape)
display(raw_1424.head(8))

# 2) 헤더 후보 행 탐지: '범죄별' 포함 + 2014 포함
header_idx = None
for i in range(min(80, len(raw_1424))):
    row = raw_1424.iloc[i].astype(str).tolist()
    joined = " ".join(row)
    if ("범죄별" in joined) and re.search(r"2014", joined):
        header_idx = i
        break

if header_idx is None:
    # fallback: 2014만 포함하는 행이라도 찾기
    for i in range(min(80, len(raw_1424))):
        row = raw_1424.iloc[i].astype(str).tolist()
        joined = " ".join(row)
        if re.search(r"2014", joined):
            header_idx = i
            break

if header_idx is None:
    raise ValueError("Header row not found (no row with '2014' within first 80 rows).")

# 3) 컬럼 재지정
header = raw_1424.iloc[header_idx].tolist()
df_1424 = raw_1424.iloc[header_idx + 1:].copy()
df_1424.columns = header
df_1424 = df_1424.reset_index(drop=True)

print("header_idx:", header_idx)
print("df_1424 columns (first 20):", list(df_1424.columns)[:20])
display(df_1424.head(10))

# 4) 연도 컬럼 탐지(강건): 컬럼명 어디든 2014~2024 숫자가 있으면 잡기
def extract_year(colname: str):
    s = str(colname)
    m = re.search(r"(20(1[4-9]|2[0-4]))", s)
    return int(m.group(1)) if m else None

year_cols_1424 = [c for c in df_1424.columns if extract_year(c) is not None]
if not year_cols_1424:
    raise ValueError("No year-like columns found even after header rectification.")

print("Detected year columns:", year_cols_1424)

# 5) 발생건수만 사용 (항목 컬럼이 있으면)
if "항목" in df_1424.columns:
    df_1424_inc = df_1424[df_1424["항목"].astype(str).str.contains("발생", na=False)].copy()
else:
    df_1424_inc = df_1424.copy()

# 6) category_raw 생성 (범죄별 + 필요 시 항목/단위는 제외)
if "범죄별" in df_1424_inc.columns:
    df_1424_inc["category_raw"] = df_1424_inc["범죄별"].astype(str).str.strip()
else:
    # fallback: 첫 컬럼을 범죄별로 간주
    first_col = df_1424_inc.columns[0]
    df_1424_inc["category_raw"] = df_1424_inc[first_col].astype(str).str.strip()

# 7) long 변환
long_1424 = df_1424_inc.melt(
    id_vars=["category_raw"],
    value_vars=year_cols_1424,
    var_name="year",
    value_name="value"
)

long_1424["year"] = long_1424["year"].astype(str).apply(extract_year).astype(int)
long_1424["value"] = pd.to_numeric(long_1424["value"], errors="coerce").fillna(0)

print("long_1424:", long_1424.shape, "| years:", long_1424["year"].min(), "-", long_1424["year"].max())
display(long_1424.head(10))


raw_1424 shape: (2234, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,○ 범죄의 발생 검거상황(총괄) [],Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
1,범죄별,항목,단위,2014 년,2015 년,2016 년,2017 년,2018 년,2019 년,2020 년,2021 년,2022 년,2023 년,2024 년
2,총계,발생건수[건],건,1933835,2020731,2008290,1824876,1738190,1767684,1714579,1531705,1575007,1613754,1729975
3,총계,발생비[건/10만명],건/10만명,3768,3921.5,3884.8,3524.4,3353.9,3409.2,3308.1,2966.2,3061.9,3144.2,3377.7
4,총계,검거건수[건],건,1518792,1638549,1691370,1556963,1466406,1479904,1399428,1228452,1219006,1272642,1363515
5,총계,발생건수대비 검거건수[%],%,78.5,81.1,84.2,85.3,84.4,83.7,81.6,80.2,77.4,78.9,78.8
6,총계,검거인원[명],명,1879548,1948966,2020196,1861796,1749459,1754808,1638387,1359952,1359389,1360807,1405185
7,총계,남자검거인원[명],명,1515159,1559659,1621184,1485212,1376363,1365413,1273827,1054864,1053321,1051171,1074546


header_idx: 1
df_1424 columns (first 20): ['범죄별', '항목', '단위', '2014 년', '2015 년', '2016 년', '2017 년', '2018 년', '2019 년', '2020 년', '2021 년', '2022 년', '2023 년', '2024 년']


,범죄별,항목,단위,2014 년,2015 년,2016 년,2017 년,2018 년,2019 년,2020 년,2021 년,2022 년,2023 년,2024 년
0,총계,발생건수[건],건,1933835,2020731,2008290,1824876,1738190,1767684,1714579,1531705,1575007,1613754,1729975
1,총계,발생비[건/10만명],건/10만명,3768,3921.5,3884.8,3524.4,3353.9,3409.2,3308.1,2966.2,3061.9,3144.2,3377.7
2,총계,검거건수[건],건,1518792,1638549,1691370,1556963,1466406,1479904,1399428,1228452,1219006,1272642,1363515
3,총계,발생건수대비 검거건수[%],%,78.5,81.1,84.2,85.3,84.4,83.7,81.6,80.2,77.4,78.9,78.8
4,총계,검거인원[명],명,1879548,1948966,2020196,1861796,1749459,1754808,1638387,1359952,1359389,1360807,1405185
5,총계,남자검거인원[명],명,1515159,1559659,1621184,1485212,1376363,1365413,1273827,1054864,1053321,1051171,1074546
6,총계,여자검거인원[명],명,336748,353977,368645,350251,344739,362614,338597,288315,287237,287597,297342
7,총계,미상검거인원[명],명,27641,35330,30367,26333,28357,26781,25963,16773,18831,22039,33297
8,총계,법인[개],개,21471,20839,22246,23850,23441,26095,22048,19747,20745,19686,15212
9,형법범,발생건수[건],건,1016209,1047761,1005689,958865,988398,1041395,1044438,917787,1003883,1026799,1096129


Detected year columns: ['2014 년', '2015 년', '2016 년', '2017 년', '2018 년', '2019 년', '2020 년', '2021 년', '2022 년', '2023 년', '2024 년']
long_1424: (8184, 3) | years: 2014 - 2024


,category_raw,year,value
0,총계,2014,1933835.0
1,총계,2014,3768.0
2,총계,2014,78.5
3,형법범,2014,1016209.0
4,형법범,2014,1980.0
5,형법범,2014,69.5
6,재산범죄,2014,617119.0
7,재산범죄,2014,1202.0
8,재산범죄,2014,56.2
9,절도,2014,266784.0


### Cell 7 - Build long_all and set CORE/EXT flags

2005–2013과 2014–2024를 결합하고 분석 구간 플래그를 부여한다


In [14]:

import pandas as pd
import numpy as np

long_all = pd.concat([long_0513, long_1424], ignore_index=True)
long_all["category_raw"] = long_all["category_raw"].astype(str).str.strip()
long_all["year"] = pd.to_numeric(long_all["year"], errors="coerce").astype(int)
long_all["value"] = pd.to_numeric(long_all["value"], errors="coerce").fillna(0)

long_all["period_flag"] = np.where(long_all["year"] <= 2023, "CORE", "EXT")
print("long_all:", long_all.shape, "| years:", long_all["year"].min(), "-", long_all["year"].max())
display(long_all.head(10))


long_all: (9984, 4) | years: 2005 - 2024


,category_raw,year,value,period_flag
0,범죄별(1) / 범죄별(2) / 범죄별(3),2005,0.0,CORE
1,형법범 / 소계 / 소계,2005,825840.0,CORE
2,형법범 / 재산범죄 / 소계,2005,442015.0,CORE
3,형법범 / 재산범죄 / 절도,2005,191114.0,CORE
4,형법범 / 재산범죄 / 장물,2005,3547.0,CORE
5,형법범 / 재산범죄 / 사기,2005,203697.0,CORE
6,형법범 / 재산범죄 / 횡령,2005,25412.0,CORE
7,형법범 / 재산범죄 / 배임,2005,5901.0,CORE
8,형법범 / 재산범죄 / 손괴,2005,12344.0,CORE
9,형법범 / 강력범죄(흉악) / 소계,2005,19941.0,CORE


### Cell 8 - Auto-generate mapping v1/v2 from actual category_raw

실제 category_raw 유니버스를 기준으로 매핑 파일을 자동 생성한다(이 셀은 long_all 변경 시 반드시 재실행)


In [15]:

import pandas as pd
import numpy as np
import re

agg = (long_all.groupby(["period_flag","category_raw"], as_index=False)["value"]
       .sum()
       .sort_values(["period_flag","value"], ascending=[True, False]))

def suggest_std(cat: str) -> str:
    s = str(cat)

    # DRUG
    if re.search(r"마약|대마|향정|마약류|필로폰|코카인|아편|환각", s):
        return "DRUG"

    # SMUGGLING / customs / trade violations
    if re.search(r"밀수|관세|통관|수출입|외국환|무역|원산지|위조지폐반입", s):
        return "SMUGGLING"

    # IMMIGRATION / border / documents
    if re.search(r"출입국|불법입국|불법체류|체류|여권|비자|국적|난민|브로커", s):
        return "IMMIGRATION"

    # TRAFFICKING / exploitation
    if re.search(r"인신|성매매|성착취|착취|알선|유인|국외이송|감금|노동착취|인신매매", s):
        return "TRAFFICKING"

    # CYBER / info-comm
    if re.search(r"사이버|해킹|피싱|랜섬|악성코드|정보통신망|전기통신|컴퓨터|전자금융|전산", s):
        return "CYBER"

    # FRAUD / financial crime
    if re.search(r"사기|보이스피싱|금융|유사수신|투자|증권|대부|카드|가상자산|코인|자금세탁|횡령|배임", s):
        return "FRAUD_FIN"

    return "OTHER"

def suggest_conf(std: str) -> str:
    return "LOW" if std == "OTHER" else "HIGH"

agg["category_std"] = agg["category_raw"].apply(suggest_std)
agg["confidence"] = agg["category_std"].apply(suggest_conf)
agg["mapping_version"] = np.where(agg["period_flag"].eq("CORE"), "v1", "v2")
agg["source"] = "NPA"
agg["year_from"] = np.where(agg["period_flag"].eq("CORE"), 2005, 2024)
agg["year_to"]   = np.where(agg["period_flag"].eq("CORE"), 2023, 2025)
agg["notes"] = "AUTO_FROM_RAW"

cols = ["mapping_version","source","year_from","year_to","category_raw","category_std","confidence","notes"]
v1 = agg[agg["period_flag"].eq("CORE")][cols].copy()
v2 = agg[agg["period_flag"].eq("EXT")][cols].copy()

v1_path = MAP_DIR / "category_mapping_v1.csv"
v2_path = MAP_DIR / "category_mapping_v2.csv"
v1.to_csv(v1_path, index=False, encoding="utf-8-sig")
v2.to_csv(v2_path, index=False, encoding="utf-8-sig")

print("Saved:")
print(" -", v1_path, "rows:", len(v1))
print(" -", v2_path, "rows:", len(v2))

# High-impact OTHER review list
TOP_N = 50
review = (agg[agg["category_std"].eq("OTHER")]
          .sort_values("value", ascending=False)
          .head(TOP_N)
          [["period_flag","category_raw","value","category_std","confidence"]]
          .copy())
review_path = MAP_DIR / "review_list_high_impact_OTHER.csv"
review.to_csv(review_path, index=False, encoding="utf-8-sig")
print(" -", review_path, "rows:", len(review))
display(review.head(20))


Saved:
 - C:\Users\82102\Desktop\Structural Shifts in International Crime_South Korea (2005–2025)\notebooks\mapping\category_mapping_v1.csv rows: 439
 - C:\Users\82102\Desktop\Structural Shifts in International Crime_South Korea (2005–2025)\notebooks\mapping\category_mapping_v2.csv rows: 239
 - C:\Users\82102\Desktop\Structural Shifts in International Crime_South Korea (2005–2025)\notebooks\mapping\review_list_high_impact_OTHER.csv rows: 50


,period_flag,category_raw,value,category_std,confidence
203,CORE,총계,17763808.5,OTHER,LOW
371,CORE,형법범,10071432.5,OTHER,LOW
285,CORE,특별법범 / 소계 / 소계,9395587.0,OTHER,LOW
414,CORE,형법범 / 소계 / 소계,8422742.0,OTHER,LOW
216,CORE,특별법범,7693221.7,OTHER,LOW
180,CORE,재산범죄,6110394.5,OTHER,LOW
422,CORE,형법범 / 재산범죄 / 소계,4915784.0,OTHER,LOW
185,CORE,절도,3913165.9,OTHER,LOW
361,CORE,폭행,2967034.3,OTHER,LOW
249,CORE,특별법범 / 도로교통법（음주운전） / 소계,2574617.0,OTHER,LOW


### Cell 9 - Final mapping apply and build contract outputs

매핑(v1/v2)을 적용해 panel/shares/totals를 생성한다


In [16]:

import pandas as pd
import numpy as np

V1_PATH = MAP_DIR / "category_mapping_v1.csv"
V2_PATH = MAP_DIR / "category_mapping_v2.csv"
if not V1_PATH.exists() or not V2_PATH.exists():
    raise FileNotFoundError("Mapping files not found. Run Cell 8 first.")

map_v1 = pd.read_csv(V1_PATH, dtype=str)
map_v2 = pd.read_csv(V2_PATH, dtype=str)

for m in [map_v1, map_v2]:
    m["category_raw"] = m["category_raw"].astype(str).str.strip()
    m["category_std"] = m["category_std"].astype(str).str.strip()
    m["source"] = m["source"].astype(str).str.strip()
    m["confidence"] = m["confidence"].astype(str).str.strip()
    m["year_from"] = pd.to_numeric(m["year_from"], errors="coerce")
    m["year_to"] = pd.to_numeric(m["year_to"], errors="coerce")
    if "notes" not in m.columns:
        m["notes"] = ""
    m["notes"] = m["notes"].astype(str)

CONF_ORDER = {"HIGH": 0, "MEDIUM": 1, "LOW": 2}

def apply_mapping(df_long: pd.DataFrame, map_df: pd.DataFrame, source="NPA") -> pd.DataFrame:
    df = df_long.copy()
    cand = df.merge(map_df[map_df["source"].eq(source)], on="category_raw", how="left")
    in_range = (cand["year_from"].notna()) & (cand["year_to"].notna()) & (cand["year"] >= cand["year_from"]) & (cand["year"] <= cand["year_to"])
    cand = cand[in_range].copy()
    cand["conf_rank"] = cand["confidence"].map(lambda x: CONF_ORDER.get(str(x).strip(), 9))
    cand = cand.sort_values(["year", "category_raw", "conf_rank"])
    best = cand.drop_duplicates(subset=["year", "category_raw"], keep="first").copy()
    out = df.merge(best[["year","category_raw","category_std","confidence","notes"]], on=["year","category_raw"], how="left")
    out["category_std"] = out["category_std"].fillna("OTHER")
    out["confidence"] = out["confidence"].fillna("LOW")
    out["notes"] = out["notes"].fillna("UNMAPPED")
    return out

core = long_all[long_all["period_flag"].eq("CORE")][["year","category_raw","value"]].copy()
ext  = long_all[long_all["period_flag"].eq("EXT")][["year","category_raw","value"]].copy()

core_m = apply_mapping(core, map_v1, source="NPA")
ext_m  = apply_mapping(ext,  map_v2, source="NPA")
mapped_all = pd.concat([core_m, ext_m], ignore_index=True)

CATEGORY_STD = ["DRUG", "TRAFFICKING", "SMUGGLING", "CYBER", "FRAUD_FIN", "IMMIGRATION", "OTHER"]

panel = (mapped_all.groupby(["year","category_std"], as_index=False)["value"]
         .sum()
         .rename(columns={"value":"count"}))
panel["source_primary"] = "NPA"
panel["period_flag"] = np.where(panel["year"] <= 2023, "CORE", "EXT")
panel["notes"] = ""

years = sorted(panel["year"].unique().tolist())
full_idx = pd.MultiIndex.from_product([years, CATEGORY_STD], names=["year","category_std"])
panel = panel.set_index(["year","category_std"]).reindex(full_idx).reset_index()
panel["count"] = panel["count"].fillna(0)
panel["source_primary"] = panel["source_primary"].fillna("NPA")
panel["period_flag"] = np.where(panel["year"] <= 2023, "CORE", "EXT")
panel["notes"] = panel["notes"].fillna("")

totals = panel.groupby("year", as_index=False)["count"].sum().rename(columns={"count":"total_count"})
totals["period_flag"] = np.where(totals["year"] <= 2023, "CORE", "EXT")

shares = panel.merge(totals[["year","total_count"]], on="year", how="left")
shares["share"] = np.where(shares["total_count"] > 0, shares["count"]/shares["total_count"], 0.0)
shares["period_flag"] = np.where(shares["year"] <= 2023, "CORE", "EXT")

panel.to_csv(DATA_PROC / "panel_year_category.csv", index=False, encoding="utf-8-sig")
shares.to_csv(DATA_PROC / "shares_year_category.csv", index=False, encoding="utf-8-sig")
totals.to_csv(DATA_PROC / "totals_year.csv", index=False, encoding="utf-8-sig")

print("Saved outputs to:", DATA_PROC)
print(" - panel_year_category.csv")
print(" - shares_year_category.csv")
print(" - totals_year.csv")


Saved outputs to: C:\Users\82102\Desktop\Structural Shifts in International Crime_South Korea (2005–2025)\notebooks\data_processed
 - panel_year_category.csv
 - shares_year_category.csv
 - totals_year.csv


### Cell 10 - QC checks (quick)

기본 품질 점검: share 합, 중복, DRUG/CYBER/FRAUD_FIN 존재 여부


In [17]:

import pandas as pd
import numpy as np

panel  = pd.read_csv(DATA_PROC / "panel_year_category.csv")
shares = pd.read_csv(DATA_PROC / "shares_year_category.csv")
totals = pd.read_csv(DATA_PROC / "totals_year.csv")

print("panel:", panel.shape, "| years:", panel["year"].min(), "-", panel["year"].max())
print("shares:", shares.shape, "| years:", shares["year"].min(), "-", shares["year"].max())
print("totals:", totals.shape, "| years:", totals["year"].min(), "-", totals["year"].max())

# share sums
ssum = shares.groupby("year")["share"].sum()
bad = ssum[(ssum < 0.999) | (ssum > 1.001)]
print("bad share years:", bad.index.tolist())

# snapshot
snap = shares[shares["year"].eq(2005)][["category_std","count","share"]].sort_values("share", ascending=False)
print("snapshot (2005):")
display(snap)

# existence checks
for cat in ["DRUG","CYBER","FRAUD_FIN","SMUGGLING","IMMIGRATION","TRAFFICKING"]:
    tot = panel.loc[panel["category_std"].eq(cat),"count"].sum()
    print(f"{cat} total:", tot)


panel: (140, 6) | years: 2005 - 2024
shares: (140, 8) | years: 2005 - 2024
totals: (20, 3) | years: 2005 - 2024
bad share years: []
snapshot (2005):


,category_std,count,share
6,OTHER,4351739.0,0.943235
4,FRAUD_FIN,240315.0,0.052088
3,CYBER,6931.0,0.001502
0,DRUG,4734.0,0.001026
2,SMUGGLING,4258.0,0.000923
1,TRAFFICKING,3966.0,0.000860
5,IMMIGRATION,1689.0,0.000366


DRUG total: 165719.30000000002
CYBER total: 559154.0000000001
FRAUD_FIN total: 9713505.299999999
SMUGGLING total: 86397.0
IMMIGRATION total: 45450.10000000001
TRAFFICKING total: 218285.4
